<a href="https://colab.research.google.com/github/NicolasMafla/08MBID---Visualizacion-de-Datos/blob/project/usa_shooting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go

In [ ]:
def clean_gender(value):
  if pd.isnull(value):
    return np.nan
  elif value == "Male":
    return "M"
  elif value == "Female":
    return "F"
  else:
    return value

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/NicolasMafla/08MBID---Visualizacion-de-Datos/project/shooting-1982-2023.csv")
cleaned_data = df[[
    "case",
    "year", 
    "type", 
    "latitude", 
    "longitude", 
    "gender", 
    "race",
    "weapon_type",
    "weapons_obtained_legally",
    "prior_signs_mental_health_issues",
    "age_of_shooter",
    "location",
    "place",
    "total_victims",
    "injured",
    "fatalities"
  ]
].copy()
cleaned_data.replace({"-": np.nan}, inplace=True)
cleaned_data["latitude"] = cleaned_data["latitude"].astype(float)
cleaned_data["longitude"] = cleaned_data["longitude"].astype(float)
cleaned_data["year"] = cleaned_data["year"].astype(int)
cleaned_data["age_of_shooter"] = cleaned_data["age_of_shooter"].astype(float)
cleaned_data["total_victims"] = cleaned_data["total_victims"].astype(float)
cleaned_data["fatalities"] = cleaned_data["fatalities"].astype(float)
cleaned_data["injured"] = cleaned_data["injured"].astype(float)
cleaned_data["gender"] = cleaned_data["gender"].apply(clean_gender)
cleaned_data.to_csv("shootings.csv", index=False)
cleaned_data

,case,year,type,latitude,longitude,gender,race,weapon_type,weapons_obtained_legally,prior_signs_mental_health_issues,age_of_shooter,location,place,total_victims,injured,fatalities
0,Nashville religious school shooting,2023,Mass,NaN,NaN,"F (identified as transgender, per police)",NaN,"semiautomatic rifle, semiautomatic handgun",NaN,NaN,28.0,"Nashville, TN",School,6.0,1.0,6.0
1,Michigan State University shooting,2023,Mass,NaN,NaN,M,Black,semiautomatic handguns,yes,NaN,43.0,"East Lansing, Michigan",School,8.0,5.0,3.0
2,Half Moon Bay spree shooting,2023,Spree,NaN,NaN,M,Asian,semiautomatic handgun,NaN,NaN,67.0,"Half Moon Bay, California",workplace,8.0,1.0,7.0
3,LA dance studio mass shooting,2023,Mass,NaN,NaN,M,Asian,semiautomatic assault weapon (Details pending),NaN,yes,72.0,"Monterey Park, California",Other,21.0,10.0,11.0
4,Virginia Walmart shooting,2022,Mass,NaN,NaN,M,Black,semiautomatic handgun,NaN,NaN,31.0,"Chesapeake, Virginia",Workplace,12.0,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Shopping centers spree killings,1987,Spree,28.033189,-80.642970,M,white,"One rifle, one revolver, one shotgun",Yes,Yes,59.0,"Palm Bay, Florida",Other,20.0,14.0,6.0
136,United States Postal Service shooting,1986,Mass,35.667201,-97.429370,M,white,Three semiautomatic handguns,Yes,Unclear,44.0,"Edmond, Oklahoma",Workplace,21.0,6.0,15.0
137,San Ysidro McDonald's massacre,1984,Mass,32.552001,-117.043081,M,white,"One semiautomatic handgun, one rifle (assault)...",Yes,Yes,41.0,"San Ysidro, California",Other,41.0,19.0,22.0
138,Dallas nightclub shooting,1984,Mass,32.925166,-96.838676,M,white,One semiautomatic handgun,No,Yes,39.0,"Dallas, Texas",Other,7.0,1.0,6.0


In [ ]:
fig = px.scatter_mapbox(
    cleaned_data,
    lon = "longitude",
    lat = "latitude",
    size = "fatalities",
    color = "type",
    hover_name = "case",
    zoom=3
)
fig.update_layout(
    mapbox_style = "carto-positron",
    legend_title = "Tipo"
)
fig.show()

In [ ]:
print(pyo.plot(fig, include_plotlyjs=False, output_type="div"))

<div>                            <div id="a66c59d1-b0ac-46b8-bbf3-d63f8b521707" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("a66c59d1-b0ac-46b8-bbf3-d63f8b521707")) {                    Plotly.newPlot(                        "a66c59d1-b0ac-46b8-bbf3-d63f8b521707",                        [{"hovertemplate":"<b>%{hovertext}</b><br><br>type=Mass<br>fatalities=%{marker.size}<br>latitude=%{lat}<br>longitude=%{lon}<extra></extra>","hovertext":["Nashville religious school shooting","Michigan State University shooting","LA dance studio mass shooting","Virginia Walmart shooting","LGBTQ club shooting","University of Virginia shooting","Greenwood Park Mall shooting","Highland Park July 4 parade shooting","Church potluck dinner shooting","Concrete company shooting","Tulsa medical center shooting","R

In [ ]:
grouped_data = cleaned_data.groupby("year", as_index=False).agg(shootings=("case", "count"))
grouped_data = pd.merge(pd.DataFrame({"year": range(1982, 2024)}), grouped_data, on="year", how="left")
grouped_data.fillna(0, inplace=True)

In [ ]:
fig = px.line(
    grouped_data, 
    x="year", 
    y="shootings", 
    template="plotly_white",
    markers=True
)
z = np.polyfit(grouped_data["year"], grouped_data["shootings"], 2)
p = np.poly1d(z)
fig.add_trace(px.line(x=grouped_data["year"], y=p(grouped_data["year"])).data[0])
fig["data"][1]["line"]["color"] = "red"
fig.show()

In [ ]:
print(pyo.plot(fig, include_plotlyjs=False, output_type="div"))

<div>                            <div id="3a2cd855-4580-4c5d-8c57-d50b5cf0c81f" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("3a2cd855-4580-4c5d-8c57-d50b5cf0c81f")) {                    Plotly.newPlot(                        "3a2cd855-4580-4c5d-8c57-d50b5cf0c81f",                        [{"hovertemplate":"year=%{x}<br>shootings=%{y}<extra></extra>","legendgroup":"","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"","orientation":"v","showlegend":false,"x":[1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023],"xaxis":"x","y":[1.0,0.0,2.0,0.0,1.0,1.0,1.0,2.0,1.0,3.0,2.0,4.0,1.0,1.0

In [ ]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(cleaned_data.columns), align="left"),
    cells=dict(
        values=[cleaned_data[c] for c in cleaned_data.columns],
        align='left'
        )
    )
])

fig.show()

In [ ]:
print(pyo.plot(fig, include_plotlyjs=False, output_type="div"))

<div>                            <div id="4b6adf9f-b297-4097-b3c1-9efa6f43c893" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("4b6adf9f-b297-4097-b3c1-9efa6f43c893")) {                    Plotly.newPlot(                        "4b6adf9f-b297-4097-b3c1-9efa6f43c893",                        [{"cells":{"align":"left","values":[["Nashville religious school shooting","Michigan State University shooting","Half Moon Bay spree shooting","LA dance studio mass shooting","Virginia Walmart shooting","LGBTQ club shooting","University of Virginia shooting","Raleigh spree shooting","Greenwood Park Mall shooting","Highland Park July 4 parade shooting","Church potluck dinner shooting","Concrete company shooting","Tulsa medical center shooting","Robb Elementary School massacre","Buffalo supermarket massacr